<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js055_BrownianMotionLJMD2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [15]:
#@title js055_BrownianMotionLJMD2D / def exec_html_js() ... exec me first

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js055_BrownianMotionLJMD2D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js055_BrownianMotionLJMD2D
//    Copyright(C) 2017-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2017.05.03 created, last updated on 2018.11.12
//    ver 0.0.1  2019.01.11 v1, last updated on 2021.04.29
//    ver 0.0.2  2021.10.28 v2, last updated on 2021.10.28
//    ver 0.0.3  2023.04.03 v3, last updated on 2023.08.27
//
// --------------------  molecular dynamics 2D
//
//    method: velocity Verlet Algorithm
//      (1) vi = vi + (Fi/mi)*(0.5dt)
//      (2) ri = ri + vi*dt
//      (3) calculation Fi <- {r1,r2,...,rn} Fi=sum(Fij,j=1 to n),Fij=F(ri-rj)
//      (4) vi = vi + (Fi/mi)*(0.5dt)
//      goto (1)
//
//    potential: Lennard-Jones V(r) = 4*epsilon*((sigma/r)^12-(sigma/r)^6)
//           force F(r) = -dV(r)/dr = 24.0*epsilon*r6*(2.0*r6-1.0)/r, r6=(sigma/r)^6
//
// --------------------
*/

const BrownianMotionLJMD2D = (function(){ // ====================  BrownianMotionLJMD2D Module  ====================

	const g_AMU = 1.66053904e-27;		// (kg) atomic mass unit
	const g_kB = 1.380649e-23;			// (J/K) Boltzmann's constant
	const g_nMax = 1000;				// array max

	let g_molecKind1 = 5;				// 5:pollen
	let g_molecKind2 = 2;				// 2:Ar
	let g_nn1 = 1;						// number of pollen
	let g_nn2 = 200;					// numner of molecules
	let g_nParticles = g_nn1+g_nn2;		// number of particles
	let g_sysTime = 0.0;				// (s) system time
	let g_timeStep = 10.0*1.0e-15;		// (s) time step dt
	let g_xMax = 10.0E-9;				// (m) x-Box size
	let g_yMax = 10.0E-9;				// (m) y-Box size

	const g_xx = dim1( g_nMax );		// (m) x-component of i-th particle position
	const g_yy = dim1( g_nMax );		// (m) y-component of i-th particle position
	const g_vx = dim1( g_nMax );		// (m/s) x-component of i-th particle velocity
	const g_vy = dim1( g_nMax );		// (m/s) y-component of i-th particle velocity
	const g_ffx = dim1( g_nMax );		// (N) x-component of total force applied i-th particle
	const g_ffy = dim1( g_nMax );		// (N) y-component of total force applied i-th particle
	const g_mas = dim1( g_nMax );		// (kg) mass of i-th particle
	const g_kind = dimInt1( g_nMax );	// kind of i-th particle

	function dim1( n ) {
		return new Float64Array( n );
	}

	function dimInt1( n ) {
		return new Int32Array( n );
	}


	// --------------------  Lennard-Jones potential data  --------------------

	const g_LJMolec = [
		//    mass(kg)      E(J)       sigma(m)   string    color
		[   4.003*g_AMU,  10.2*g_kB, 2.576e-10, "He",  "#ee4444" ], // 0 He
		[  20.183*g_AMU,  36.2*g_kB, 2.976e-10, "Ne",  "#eeaa00" ], // 1 Ne
		[  39.948*g_AMU, 124.0*g_kB, 3.418e-10, "Ar",  "#aadd00" ], // 2 Ar
		[  83.500*g_AMU, 190.0*g_kB, 3.610e-10, "Kr",  "#44dd44" ], // 3 Kr
		[ 131.300*g_AMU, 229.0*g_kB, 4.055e-10, "Xe",  "#0088ee" ], // 4 Xe
		[1000.00 *g_AMU, 124.0*g_kB,20.000e-10, "pollen", "#ffff00" ]  // 5 pollen
	];

	const g_rCollision = [];		// (m) = sigma, V(sigma)==0.0
	const g_rBond = [];			// (m) bond length == 2.0*g_rBond[kind], == sigma*2^(1/6)
	const g_massOf = [];			// (kg) g_massOf[kind] : mass of kind
	const g_strOf = [];			// g_strOf[kind] : string of kind, such as "Ar"
	const g_colorOf = [];			// g_colorOf[kind] : color of kind

	(function() {
		const n=g_LJMolec.length;
		for (let i=0; i<n; i++) {
			g_rCollision[i] = g_LJMolec[i][2]*0.5;
			g_rBond[i] = g_LJMolec[i][2]*0.5*1.12246;
			g_massOf[i] = g_LJMolec[i][0];
			g_strOf[i] = g_LJMolec[i][3];
			g_colorOf[i] = g_LJMolec[i][4];
		}
	}());


	// --------------------  set initial condition  --------------------

	function setInitialCondition( theme, nn, contTemp ) {
		g_molecKind1 = 5; // 5:pollen
		g_molecKind2 = theme; // 2:Ar
		g_sysTime = 0.0;
		g_nParticles = setPollenAndParticles(g_molecKind2,nn+1);
		ajustVelocity(contTemp);
		setInitialPollenPos();

	}

	function setPollenAndParticles(knd2,nn) {
		let nParticles = nn;
		let j = 0;
		g_xx[j] = g_xMax/2.0; g_yy[j] = g_yMax/2.0; g_kind[j] = g_molecKind1; // pollen
		for (j=1; j<nn; j++) {
			let i;
			let loopCount = 0;
			g_kind[j] = knd2;
			do {
				const sgm = 2.0*g_rCollision[g_kind[j]];
				g_xx[j] = (g_xMax-1.2*sgm)*Math.random() + 0.6*sgm;
				g_yy[j] = (g_yMax-1.2*sgm)*Math.random() + 0.6*sgm;
				for (i=0; i<j; i++) {
					const d = Math.sqrt((g_xx[i]-g_xx[j])*(g_xx[i]-g_xx[j])+(g_yy[i]-g_yy[j])*(g_yy[i]-g_yy[j]));
					if (d < 1.2*(g_rCollision[g_kind[i]]+g_rCollision[g_kind[j]])) break;
				}
				loopCount = loopCount + 1;
				if (loopCount>1000) break;
			} while (i<j);
			if (loopCount>1000) {
				nParticles = j - 1;
				break;
			}
		}
		for (let i=0; i<nParticles; i++) {
			const v0 = 200.0/(Math.sqrt(g_massOf[g_kind[i]]/g_massOf[knd2]));
			g_vx[i] = v0*normalRandom3();
			g_vy[i] = v0*normalRandom3();
			g_ffx[i] = 0.0;
			g_ffy[i] = 0.0;
			g_mas[i] = g_massOf[g_kind[i]];
		}
		return nParticles;
	}

	// normal distributed random number: -3.0 <= normalRandom3() < 3.0
	function normalRandom3() {
		return (Math.random()+Math.random()+Math.random()+Math.random()+Math.random()+Math.random()-3.0);
	}


	// --------------------  time evolution  --------------------

	function timeEvolution( nCalc, tempMode, contTemp ) {
		for (let ii=0; ii<nCalc; ii++) {
			if (tempMode==1) ajustVelocity(contTemp);
			for (let i=1; i<=20; i++) {
				g_sysTime += g_timeStep;
				moveParticles(g_timeStep);
			}
		}
		setPollenPos();
	}

	function moveParticles(dt) {
		const nn=g_nParticles;

		for (let i=0; i<nn; i++) {
			const a = 0.5*dt/g_mas[i];
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
			g_xx[i] += g_vx[i]*dt;
			g_yy[i] += g_vy[i]*dt;
		}
		calcForce();
		for (let i=0; i<nn; i++) {
			const a = 0.5*dt/g_mas[i];
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
		}
	}

	function calcForce() {
		const nn=g_nParticles, s05 = 0.5*3.418e-10; // Ar sigma=3.418e-10
		const sp05 = 0.5*20.000e-10; // 0.5*pollen sigma
		for (let i=0; i<nn; i++) {
			g_ffx[i]=0;g_ffy[i]=0;
		}
		for (let i=0; i<nn-1; i++) {
			for (let j=i+1; j<nn; j++) {
				const xij=g_xx[i]-g_xx[j], yij=g_yy[i]-g_yy[j];
				const rij = Math.sqrt(xij*xij+yij*yij);
				const f = force(rij,g_kind[i],g_kind[j]);
				const fxij = f*xij/rij, fyij = f*yij/rij;
				g_ffx[i] += fxij;
				g_ffy[i] += fyij;
				g_ffx[j] -= fxij;
				g_ffy[j] -= fyij;
			}
		}
		for (let i=0; i<g_nn1; i++) {
			g_ffx[i] += pollenBoundaryForce(g_xx[i]+sp05)+pollenBoundaryForce(g_xx[i]-g_xMax-sp05);
			g_ffy[i] += pollenBoundaryForce(g_yy[i]+sp05)+pollenBoundaryForce(g_yy[i]-g_yMax-sp05);
		}
		for (let i=g_nn1; i<nn; i++) {
			g_ffx[i] += boundaryForce(g_xx[i]+s05)+boundaryForce(g_xx[i]-g_xMax-s05);
			g_ffy[i] += boundaryForce(g_yy[i]+s05)+boundaryForce(g_yy[i]-g_yMax-s05);
		}
	}

	function force(r,ki,kj) {
		const ep = Math.sqrt(g_LJMolec[ki][1]*g_LJMolec[kj][1]);
		const sg = 0.5*(g_LJMolec[ki][2]+g_LJMolec[kj][2]);
		const ri = (sg/r);
		const r6 =ri*ri*ri*ri*ri*ri;
		return 24.0*ep*r6*(2.0*r6-1.0)/r;
	}

	function pollenBoundaryForce(r) {
		const ri = (g_LJMolec[5][2]/r);
		const r6 =ri*ri*ri*ri*ri*ri;
		return (24.0*0.5*1.711e-21*r6*(2.0*r6-1.0)/r);
	}

	function boundaryForce(r) {
		const ri = (3.418e-10/r);
		const r6 = ri*ri*ri*ri*ri*ri;
		return (24.0*0.5*1.711e-21*r6*(2.0*r6-1.0)/r);
	}

	//--- utility

	function systemTemperature() {
		const nn=g_nParticles;
		let ek = 0.0;
		for (let i=0; i<nn; i++) {
			ek = ek + 0.5*g_mas[i]*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]);
		}
		return ek/(nn*g_kB);
	}

	function ajustVelocity(temp) {
		const nn=g_nParticles;
		const a = Math.sqrt(temp/systemTemperature());
		for (let i=0; i<nn; i++) {
			g_vx[i] = a*g_vx[i];
			g_vy[i] = a*g_vy[i];
		}
	}


	// --------------------  pollen  --------------------

	const pollenPos = [];
	pollenPos.pointer = 0;
	pollenPos.pointerMax =3000;

	function setInitialPollenPos() {
		for (let i=0; i<pollenPos.pointerMax; i++) {
			pollenPos[i] = [ g_xx[0], g_yy[0] ];
		}
	}

	function setPollenPos() {
		pollenPos[pollenPos.pointer] = [ g_xx[0], g_yy[0] ];
		pollenPos.pointer = (pollenPos.pointer+1)%pollenPos.pointerMax;
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( theme, nn, contTemp )
		evolve:			timeEvolution,			// timeEvolution( ncalc, tempMode, contTemp)

		setTemp:		ajustVelocity,			// ajustVelocity( temp )

		getSysParam:	function() { return [ g_molecKind1, g_molecKind2, g_nParticles, g_timeStep, g_xMax, g_yMax ]; },
		getPollenPos:	function() { return pollenPos; },
		getNow:			function() { return [ g_sysTime, systemTemperature() ]; },
		getStrOf:		function(kind) { return g_strOf[kind]; },
		getColorOf:		function(kind) { return g_colorOf[kind]; },
		getMolecData:	function(i) { const kind = g_kind[i]; return [ g_rCollision[kind], g_colorOf[kind] ]; },
		getPosition:	function(i) { return [ g_xx[i], g_yy[i] ]; },
		getVelocity:	function(i) { return [ g_vx[i], g_vy[i] ]; },
		getForce:		function(i) { return [ g_ffx[i], g_ffy[i] ]; },
	};

})(); // ====================  BrownianMotionLJMD2D end  ====================


const js055 = (function(){ // ====================  js Module  ====================

	const theModule = BrownianMotionLJMD2D;
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;
	let ctx;

	let v_theme = 2;			// theme  1:Ne 2:Ar 3:Kr 4:Xe 5:Hg ... 13:pollen
	let v_nn = 200;				// number of max. settable molecules
	let v_BoxSizeInNM = 12.0;
	let v_nCalc = 1;
	let v_tempMode = 0;			// 0: adiabatic, 1: temp.control
	let v_contTemp = 300.0;

	let  p_kind1, p_kind2, p_nParticles, p_timeStep, p_xMax, p_yMax // = theModule.getSysParam();
	let sysTime, temperature;
  let xxList = [];
	let yyList = [];
	let vxList = [];
	let vyList = [];
  let xPollenList = [];
  let yPollenList = [];

	let dispMode = 0;
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;
  let breakFlag = false;
  let perticleFlag = true;
  let pollenFlag = true;


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_theme, v_nn, v_contTemp ); // ( kind2, nn, contTemp )
			[ p_kind1, p_kind2, p_nParticles, p_timeStep, p_xMax, p_yMax ] = theModule.getSysParam();
      perticleFlag = true;
      pollenFlag = true;
		}

		if ( !pauseFlag ) {
				theModule.evolve( v_nCalc, v_tempMode, v_contTemp );
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( 1, v_tempMode, v_contTemp );
		}

    if ( perticleFlag ) setParticlesData();
    if ( pollenFlag ) setPollenTrace();

		draw( ctx, dispMode );

		requestAnimationFrame(animate);
	}

  function setParticlesData() {
    for (let i=0; i<p_nParticles; i++) {
			let x, y, vx, vy;
			[ x, y ] = theModule.getPosition(i);
			xxList[i] = x;
			yyList[i] = y;
			[ vx, vy ] = theModule.getVelocity(i);
			vxList[i] = vx;
			vyList[i] = vy;
		}
	}

	function setPollenTrace() {
		const pollenPos = theModule.getPollenPos();
		const nn = pollenPos.pointerMax, pp = pollenPos.pointer;

		for (let i=0; i<nn; i++) {
			const j = (pp+i)%nn;
      const x = pollenPos[j][0];
			const y = pollenPos[j][1];
      xPollenList[i] = x;
      yPollenList[i] = y;
		}
	}


	// --------------------  draw  --------------------

	function draw( ctx, dispMode ) {
		const xp = 40, yp = 5, xSize = 400, ySize = 400, yTextPos= 430;
		const scale = xSize/p_xMax

		// clear
		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		// box
		ctx.strokeStyle = "#888800";
		ctx.strokeRect( xp, yp, xSize, ySize );

		if (dispMode==0) {
			drawBalls(ctx, p_nParticles, p_yMax, xp, yp, scale );
		} else if (dispMode==1) {
			drawBalls(ctx, 1, p_yMax, xp, yp, scale );
		}  else if (dispMode==2) {
			drawBalls(ctx, 1, p_yMax, xp, yp, scale );
			drawPollenTrace( ctx, p_yMax, xp, yp, scale );
		}

		// caption
		[ sysTime, temperature ] = theModule.getNow();
		const molecStr = theModule.getStrOf(p_kind2);
		ctx.fillStyle = "#888888";
		ctx.fillText(`time = "${(sysTime*1.0e12).toFixed(1)} (ps)`, xp, yTextPos);
		ctx.fillText(`Temp = ${temperature.toFixed(1)} (k)`, xp+200, yTextPos);
		ctx.fillText(`molec:${molecStr}, N = ${p_nParticles-1}`, xp, yTextPos+20);
		ctx.fillText(`cont. Temp = ${v_contTemp.toFixed(0)} (K)`, xp+200, yTextPos+20);
		ctx.fillText(`Box = ${(p_xMax*1.0e9).toFixed(1)}x${(p_yMax*1.0e9).toFixed(1)} (nm)`, xp, yTextPos+40);
		//document.getElementById("text_caption").innerHTML = "minimum molecular dynamics code"
	}

	function drawBalls( ctx, nn, yMax, xp, yp, scale ) {
		const th = 2.0*Math.PI;
		for (let i=0; i<nn; i++) {
			let x, y, r, color;
			[ x, y ] = theModule.getPosition(i);
			[ r, color ] = theModule.getMolecData(i);
			ctx.fillStyle = color;
			ctx.beginPath();
			ctx.arc(x*scale+xp, (yMax-y)*scale+yp, r*scale, 0, th, false);
			ctx.fill();
		}
	}

	function drawPollenTrace( ctx, yMax, xp, yp, scale ) {
		const pollenPos = theModule.getPollenPos();
		const nn = pollenPos.pointerMax, pp = pollenPos.pointer;
		//document.getElementById("text_caption").innerHTML = `nn=${nn}, pp=${pp}`;
		ctx.strokeStyle = "#ffaaaa";
		ctx.beginPath();
		for (let i=0; i<nn; i++) {
			const j = (pp+i)%nn;
        	const x = pollenPos[j][0];
			const y = pollenPos[j][1];
			ctx.lineTo( x*scale+xp, (yMax-y)*scale+yp );
		}
		ctx.stroke();
	}


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {
		v_theme = 1 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}

	function setTempMode() {
		v_tempMode = 0 + document.getElementById("slct_tempMode").selectedIndex;
	}

	function setContTemp() {  // range mixing
		v_contTemp = Number(document.getElementById("range_temp").value);
		document.getElementById("text_temp").innerHTML = " " + v_contTemp.toFixed(0);
	}

	function setDispMode() {
		dispMode = 0 + document.getElementById("slct_disp").selectedIndex;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( theme ) {
    v_theme = 1 + theme
    document.getElementById("slct_theme").selectedIndex = theme;
    resetFlag = true;
  }

  function pysetTempMode( mode ) {
    v_tempMode = mode;
    document.getElementById("slct_tempMode").selectedIndex = mode;
  }

  function pysetTemperature(temp) {
    theModule.setTemp(temp);
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ sysTime, temperature ];
  }

  function pygetParticlesList() {
    perticleFlag = false;
    return [ xxList, yyList, vxList, vyList ];
  }

  function pygetPollenList() {
    pollenFlag = false;
    return [ xPollenList, yPollenList ];
  }


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		// dom control
		reset:			reset,			// reset()
		pause:			pause,			// pause()
		step:			step,			// step()

		setTheme:		setTheme,		// setTheme()
		setTempMode:	setTempMode,	// setTempMode()
		setContTemp:	setContTemp,	// setContTemp()
		setDispMode:	setDispMode,	// setDispMode()

    breakLoop: breakLoop, // breakLoop();
    pysetTheme: pysetTheme, // pysetTheme( theme )
    pysetTempMode: pysetTempMode, // pysetTempMode( mode )
    pysetTemperature: pysetTemperature, // pysetTemperature( temp )
    pysetDispMode: pysetDispMode, // pysetDispMode( mode )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ sysTime, temperature ]
    pygetParticlesList, pygetParticlesList, //() :return [ xxList, yyList, vxList, vyList ]
    pygetPollenList: pygetPollenList, // pygetPollenList() return [ xPollenList, xPollenList ]
	};

})(); // ====================  js055 module end  ====================


const js = js055;
//window.addEventListener('load', js.main );
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js055] Brownian motion - Lennard-Jones molecular dynamics 2D</p>
<canvas ID="canvas_box" style="background-color: #000000;" width="480" height="480"></canvas>
<br>

<label>molecule:</label>
<select id="slct_theme" onChange="js.setTheme()">
<option>Ne</option><option selected>Ar</option><option>Kr</option><option>Xe</option>
</select>
    <span style="margin-right: 200px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>temp mode:</label>
<select id="slct_tempMode" onChange="js.setTempMode()">
<option selected>adiabatic</option><option>Temp.control</option>
</select>
<br>

<label>cont. Temp</label><label id="text_temp"> 300</label>
<input type="range" id="range_temp" min="10" max="600" value="300" step="2"
style="width:360px" oninput="js.setContTemp()">
<br>

<label>disp mode:</label>
<select id="slct_disp" onChange="js.setDispMode()">
<option selected>molecules + pollen</option>
<option>pollen only</option>
<option>pollen trace</option>
</select>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# exec html-js code, and python control

import time

# exec html-js code
exec_html_js()
print("-- start --")
time.sleep(3)

# python control
for i in range(10):
  [ sysTime, sysTemp ] = eval_js( 'js.pygetData({})'.format(i) )
  print( f'i = {i:>2d},  time = {sysTime*1e12:>7.2f} (ps), temp = {sysTemp:>6.1f} (K)' )
  time.sleep(3)

time.sleep(1)
# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# exec html-js code, and python control / change theme

import time

# exec html-js code
exec_html_js()
print("-- start --")
time.sleep(3)

# reset
eval_js( 'js.reset()' )
print("-- reset --")

# change theme
themeList = [ 'Ne', 'Ar', 'Kr', 'Xe' ]
for theme in [ 0, 1, 2, 3 ]:
  eval_js( 'js.pysetTheme({})'.format(theme) )
  print( "-- theme:", themeList[theme], "--" )
  for i in range(2):
    [ sysTime, sysTemp ] = eval_js( 'js.pygetData({})'.format(i) )
    print( f'\t i = {i:>2d},  time = {sysTime*1e12:>7.2f} (ps), temp = {sysTemp:>6.1f} (K)' )
    time.sleep(3)

time.sleep(1)
# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# exec html-js code, and python control / plot vx, vy ( velocity space )

import time
import matplotlib.pyplot as plt

# exec html-js code
exec_html_js()
print("-- start --")

# get data and print
for i in range(10):
  [ sysTime, sysTemp ] = eval_js( 'js.pygetData({})'.format(i) )
  print( f'i = {i:>2d},  time = {sysTime*1e12:>7.2f} (ps), temp = {sysTemp:>6.1f} (K)' )
  time.sleep(1)

# get particle dara and plot
[ xxList, yyList, vxList, vyList ] = eval_js( 'js.pygetParticlesList()' )
print("-- plot vx, xy --", ", N = ", len(vxList) )
plt.figure(figsize=(6,6))
plt.title("velocity space")
plt.scatter( vxList, vyList )
plt.xlabel("vx")
plt.ylabel("vy")
plt.grid()
plt.show()

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# exec html-js code, and python control / plot pllen trace

import time
import numpy as np
import matplotlib.pyplot as plt

# exec html-js code
exec_html_js()
print("-- start --")

# get data and print
for i in range(10):
  [ sysTime, sysTemp ] = eval_js( 'js.pygetData({})'.format(i) )
  print( f'i = {i:>2d},  time = {sysTime*1e12:>7.2f} (ps), temp = {sysTemp:>6.1f} (K)' )
  time.sleep(1)

# get particle dara and plot
[ xPollenList, yPollenList ] = eval_js( 'js.pygetPollenList()' )
X = np.array(xPollenList)*1e9
Y = np.array(yPollenList)*1e9
print("-- pollen trace, N = ", len(xPollenList) )
plt.figure(figsize=(6,6))
plt.title("pollen trace")
plt.plot( X, Y, '-' )
plt.xlabel("x (nm)")
plt.ylabel("y (nm)")
plt.xlim(0,10)
plt.ylim(0,10)
plt.grid()
plt.show()

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")